# __CONVERTING NEXT DAY WILDFIRE DATASET TF RECORDS TO CSV__

In [65]:
# importing libraries

import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pandas_tfrecords as pdtfr


In [66]:
# checking filenames and setting dataset path
dataset_path = '../../datasets/next_day_wildfire/'

save_path = "../../datasets/next_day_wildfire_csv/"

for subdirs, dirs, files in os.walk(dataset_path):
    for file in files:
      print(file)

next_day_wildfire_spread_train_03.tfrecord
next_day_wildfire_spread_train_09.tfrecord
next_day_wildfire_spread_train_14.tfrecord
next_day_wildfire_spread_train_11.tfrecord
next_day_wildfire_spread_train_04.tfrecord
next_day_wildfire_spread_test_01.tfrecord
next_day_wildfire_spread_train_10.tfrecord
next_day_wildfire_spread_train_08.tfrecord
next_day_wildfire_spread_eval_00.tfrecord
next_day_wildfire_spread_train_06.tfrecord
next_day_wildfire_spread_train_00.tfrecord
next_day_wildfire_spread_train_02.tfrecord
next_day_wildfire_spread_train_05.tfrecord
next_day_wildfire_spread_eval_01.tfrecord
next_day_wildfire_spread_train_01.tfrecord
next_day_wildfire_spread_train_07.tfrecord
next_day_wildfire_spread_train_13.tfrecord
next_day_wildfire_spread_test_00.tfrecord
next_day_wildfire_spread_train_12.tfrecord


In [67]:
def add_headings(tfrecord_file):
    # Replace 'your_file.tfrecord' with your actual TFRecord file.
    # tfrecord_file = 'next_day_wildfire_spread_eval_00.tfrecord'
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    
    headings = None
    
    for record in dataset:
        example = tf.train.Example.FromString(record.numpy())
        
        # Assuming that the first record contains column names.
        if headings is None:
            headings = list(example.features.feature.keys())
        else:
            break
    
    return headings
    

In [68]:
def f_parser(tfrecord_file, headings):
    # Replace 'your_file.tfrecord' with your actual TFRecord file.
    # tfrecord_file = 'next_day_wildfire_spread_eval_00.tfrecord'
    dataset = tf.data.TFRecordDataset(tfrecord_file)

    # Create an empty list to hold the data.
    data = []
    
    data.append(headings)
    
    # Initialize a list to store the column names.

    # Iterate through the TFRecord file.
    for record in dataset:
        example = tf.train.Example.FromString(record.numpy())
        
        current_data = {}
        
        # Iterate through the features and handle their types.
        for key, feature in example.features.feature.items():
            if feature.HasField('bytes_list'):
                # Handle string data.
                current_data[key] = feature.bytes_list.value[0].decode('utf-8')
            elif feature.HasField('int64_list'):
                # Handle integer data.
                current_data[key] = feature.int64_list.value[0]
            elif feature.HasField('float_list'):
                # Handle float data.
                current_data[key] = feature.float_list.value[0]
        
        current_example = []
        for heading in headings:
            current_example.append(current_data[heading])
        
        # Append the data for the current example.
        data.append(current_example)

    # Save the data as a CSV file
    file_name = tfrecord_file.replace(".tfrecord", "") + '.csv' 
    csv_filename = file_name.split("/")
    csv_filename = csv_filename[-1]
    # CSV Dataset Path Added
    
    
    np.savetxt(save_path + csv_filename, data, delimiter=',', fmt='%s')
    return csv_filename

In [69]:
headings = None

headings = add_headings(dataset_path + "next_day_wildfire_spread_train_00.tfrecord")

print(headings)

headings = sorted(headings)

print(headings)

['NDVI', 'tmmn', 'elevation', 'population', 'FireMask', 'vs', 'pdsi', 'pr', 'tmmx', 'sph', 'th', 'PrevFireMask', 'erc']
['FireMask', 'NDVI', 'PrevFireMask', 'elevation', 'erc', 'pdsi', 'population', 'pr', 'sph', 'th', 'tmmn', 'tmmx', 'vs']


In [72]:
os.mkdir(save_path)

# print("HEADINGS", headings)

for subdirs, dirs, files in os.walk(dataset_path):
    # This function will gene
    for file in files:
        # if (file[-4:] == ".csv"):
            # os.remove(dataset_path + file)
        csv_filename = f_parser(dataset_path + file, headings)
        

In [73]:
print(headings)

['tmmn', 'NDVI', 'elevation', 'population', 'FireMask', 'pdsi', 'vs', 'pr', 'tmmx', 'sph', 'th', 'PrevFireMask', 'erc']


In [71]:
### WARNING ###
# THIS CELL IS USED TO DELETE THE CSV FOLDER #

for subdirs, dirs, files in os.walk(save_path):
    # This function will gene
    for file in files:
        os.remove(save_path + file)

os.rmdir(save_path)